In [2]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, BertTokenizer, BertModel, BertForMaskedLM
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm

In [3]:
#Load the data
train_df = pd.read_csv("propaganda_train.tsv", delimiter='\t')
test_df = pd.read_csv("propaganda_val.tsv", delimiter='\t')

In [4]:
train_df.head()

,label,tagged_in_context
0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed."
1,not_propaganda,This declassification effort <BOS> won’t make ...
2,flag_waving,The Obama administration misled the <BOS> Amer...
3,not_propaganda,“It looks like we’re capturing the demise of t...
4,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>"


In [5]:
test_df.head()

,label,tagged_in_context
0,not_propaganda,"On average, between 300 and 600 infections are..."
1,causal_oversimplification,Mostly because <BOS> the country would not las...
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.
4,repetition,It must be exacted from him directly in order ...


In [6]:
# check for null values
train_df.isnull().sum()

label                0
tagged_in_context    0
dtype: int64

In [7]:
# check for null values
test_df.isnull().sum()

label                0
tagged_in_context    0
dtype: int64

In [8]:
#Size of the data
print("Training data shape:", train_df.shape)
print("Test data shape:", test_df.shape)

Training data shape: (2414, 2)
Test data shape: (580, 2)


In [9]:
#Number of instances for each label
print("Training data label distribution:")
print(train_df["label"].value_counts())

Training data label distribution:
not_propaganda               1191
exaggeration,minimisation     164
causal_oversimplification     158
name_calling,labeling         157
loaded_language               154
appeal_to_fear_prejudice      151
flag_waving                   148
repetition                    147
doubt                         144
Name: label, dtype: int64


In [10]:
#Percentage of each label
print("Training data label percentages:")
print(train_df["label"].value_counts(normalize=True)*100)

Training data label percentages:
not_propaganda               49.337200
exaggeration,minimisation     6.793703
causal_oversimplification     6.545153
name_calling,labeling         6.503728
loaded_language               6.379453
appeal_to_fear_prejudice      6.255178
flag_waving                   6.130903
repetition                    6.089478
doubt                         5.965203
Name: label, dtype: float64


## 1: Propaganda vs Not Propaganda
### Approach 1: Fine-tuned BERT

In [11]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader

# Load the data
train_df = pd.read_csv("propaganda_train.tsv", delimiter='\t')
test_df = pd.read_csv("propaganda_val.tsv", delimiter='\t')

# Split the training data into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Preprocess the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(data, max_length=128):
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in data.iterrows():
        encoded_dict = tokenizer.encode_plus(
            row['tagged_in_context'],
            add_special_tokens=True,
            max_length=64,
            padding="max_length",
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        labels.append(1 if row['label'] != 'not_propaganda' else 0)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

train_input_ids, train_attention_masks, train_labels = preprocess_data(train_df)
val_input_ids, val_attention_masks, val_labels = preprocess_data(val_df)
test_input_ids, test_attention_masks, test_labels = preprocess_data(test_df)

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)

# Fine-tune the model
batch_size = 32
epochs = 3

train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = torch.utils.data.RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_sampler = torch.utils.data.SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_sampler = torch.utils.data.SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)

    for batch in progress_bar:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch

        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': loss.item()})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} - Average training loss: {avg_train_loss:.4f}")

    model.eval()
    val_preds = []
    val_true_labels = []

    for batch in val_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_masks)

        logits = outputs.logits
        batch_preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
        batch_true_labels = labels.detach().cpu().numpy()

        val_preds.extend(batch_preds)
        val_true_labels.extend(batch_true_labels)

    # Calculate validation accuracy before mapping predicted values to labels
    val_accuracy = np.mean(np.array(val_preds) == np.array(val_true_labels))
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    
    # Map actual labels to labels
    actual_labels_mapped = ['not_propaganda' if label == 0 else 'propaganda' for label in val_true_labels]

    # Map predicted values to labels
    predicted_labels = ['not_propaganda' if pred == 0 else 'propaganda' for pred in val_preds]

    # Create pandas Series for predicted and actual labels
    actual_labels_series = pd.Series(actual_labels_mapped, name='Actual')
    val_preds_series = pd.Series(predicted_labels, name='Predicted')

    # Concatenate the actual and predicted labels into a DataFrame
    results_df = pd.concat([actual_labels_series, val_preds_series], axis=1)

    print(results_df)

# Classification report
print(classification_report(actual_labels_series, val_preds_series))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3 - Average training loss: 0.5428
Validation Accuracy: 0.8965
             Actual       Predicted
0    not_propaganda  not_propaganda
1    not_propaganda  not_propaganda
2        propaganda      propaganda
3        propaganda      propaganda
4        propaganda      propaganda
..              ...             ...
478      propaganda      propaganda
479      propaganda      propaganda
480      propaganda  not_propaganda
481  not_propaganda  not_propaganda
482      propaganda  not_propaganda

[483 rows x 2 columns]


Epoch 2/3 - Average training loss: 0.2917
Validation Accuracy: 0.9006
             Actual       Predicted
0    not_propaganda  not_propaganda
1    not_propaganda  not_propaganda
2        propaganda      propaganda
3        propaganda      propaganda
4        propaganda      propaganda
..              ...             ...
478      propaganda      propaganda
479      propaganda      propaganda
480      propaganda  not_propaganda
481  not_propaganda  not_propaganda
482      propaganda  not_propaganda

[483 rows x 2 columns]


Epoch 3/3 - Average training loss: 0.1829
Validation Accuracy: 0.9337
             Actual       Predicted
0    not_propaganda  not_propaganda
1    not_propaganda  not_propaganda
2        propaganda      propaganda
3        propaganda      propaganda
4        propaganda      propaganda
..              ...             ...
478      propaganda      propaganda
479      propaganda      propaganda
480      propaganda  not_propaganda
481  not_propaganda      propaganda
482      propaganda  not_propaganda

[483 rows x 2 columns]
                precision    recall  f1-score   support

not_propaganda       0.94      0.93      0.93       241
    propaganda       0.93      0.94      0.93       242

      accuracy                           0.93       483
     macro avg       0.93      0.93      0.93       483
  weighted avg       0.93      0.93      0.93       483



#### Model Testing

In [12]:
# Define a function for testing
def test_model(model, test_dataloader):
    model.eval()
    test_preds = []
    test_true_labels = []

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_masks)

        logits = outputs.logits
        batch_preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
        batch_true_labels = labels.detach().cpu().numpy()

        test_preds.extend(batch_preds)
        test_true_labels.extend(batch_true_labels)

    # Calculate test accuracy before mapping predicted values to labels
    test_accuracy = np.mean(np.array(test_preds) == np.array(test_true_labels))
    print(f"Test Accuracy: {test_accuracy:.4f}")

    # Map actual labels to labels
    actual_labels_mapped = ['not_propaganda' if label == 0 else 'propaganda' for label in test_true_labels]

    # Map predicted values to labels
    predicted_labels = ['not_propaganda' if pred == 0 else 'propaganda' for pred in test_preds]

    # Create pandas Series for predicted and actual labels
    actual_labels_series = pd.Series(actual_labels_mapped, name='Actual')
    test_preds_series = pd.Series(predicted_labels, name='Predicted')

    # Concatenate the actual and predicted labels into a DataFrame
    results_df = pd.concat([actual_labels_series, test_preds_series], axis=1)

    print(results_df)

    # Classification report for test dataset
    print(classification_report(actual_labels_series, test_preds_series))

# Call the function to test the model
test_model(model, test_dataloader)


Test Accuracy: 0.9103
             Actual       Predicted
0    not_propaganda  not_propaganda
1        propaganda      propaganda
2        propaganda      propaganda
3    not_propaganda  not_propaganda
4        propaganda      propaganda
..              ...             ...
575  not_propaganda  not_propaganda
576  not_propaganda  not_propaganda
577  not_propaganda  not_propaganda
578      propaganda      propaganda
579      propaganda      propaganda

[580 rows x 2 columns]
                precision    recall  f1-score   support

not_propaganda       0.94      0.89      0.91       301
    propaganda       0.88      0.94      0.91       279

      accuracy                           0.91       580
     macro avg       0.91      0.91      0.91       580
  weighted avg       0.91      0.91      0.91       580



### Approach 2: Convolutional Neural Network (CNN) with GloVe Embeddings

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

2024-05-13 11:42:20.260512: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Preprocess the text data
def preprocess_text(text):
    return text.lower()

train_df["text"] = train_df["tagged_in_context"].apply(preprocess_text)
test_df["text"] = test_df["tagged_in_context"].apply(preprocess_text)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(train_df["text"], 
                                                                    train_df["label"], 
                                                                    test_size=0.2, 
                                                                    random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

# Pad the sequences to a fixed length
max_length = 100
train_data = pad_sequences(train_sequences, maxlen=max_length)
val_data = pad_sequences(val_sequences, maxlen=max_length)

# Prepare the labels
train_labels = np.where(train_labels == "not_propaganda", 0, 1)
val_labels = np.where(val_labels == "not_propaganda", 0, 1)

# Load the GloVe word embeddings
embedding_dim = 100
glove_file = "glove.6B.100d.txt"  # Download the GloVe embeddings file

embeddings_index = {}
with open(glove_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs

# Create the embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build the CNN model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(128, 5, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
epochs = 5
batch_size = 32
history = model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=epochs, batch_size=batch_size)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_data, val_labels, batch_size=batch_size)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

# Predict labels for validation data
predictions = model.predict(val_data)
predicted_labels = np.round(predictions).astype(int).flatten()

# Map actual labels to labels
actual_labels_mapped = ['not_propaganda' if label == 0 else 'propaganda' for label in val_labels]

# Map predicted values to labels
predicted_labels_mapped = ['not_propaganda' if pred == 0 else 'propaganda' for pred in predicted_labels]

# Convert predicted labels and validation labels to pandas Series
val_labels_series = pd.Series(actual_labels_mapped, name='Actual')
predicted_labels_series = pd.Series(predicted_labels_mapped, name='Predicted')

# Concatenate the actual and predicted labels into a DataFrame
results_df = pd.concat([val_labels_series, predicted_labels_series], axis=1)

print(results_df.head(10))

# Classification report
print(classification_report(val_labels, predicted_labels))


Epoch 1/5


/Users/maicymaritim/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6019 - loss: 0.7063 - val_accuracy: 0.7930 - val_loss: 0.4443
Epoch 2/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8376 - loss: 0.3913 - val_accuracy: 0.8427 - val_loss: 0.3554
Epoch 3/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8933 - loss: 0.2760 - val_accuracy: 0.8571 - val_loss: 0.3455
Epoch 4/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9310 - loss: 0.1773 - val_accuracy: 0.8592 - val_loss: 0.3487
Epoch 5/5
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9725 - loss: 0.1053 - val_accuracy: 0.8116 - val_loss: 0.4758
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7677 - loss: 0.5930
Validation Loss: 0.4758431017398834
Validation Accuracy: 0.8115941882133484
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
           Actual       Predicted
0  not_propaganda  not_propaganda
1  not_propaganda      propaganda
2      propaganda      propaganda
3      propaganda      propaganda
4      propaganda  no

#### Model Testing

In [15]:
# Preprocess the text data
test_df["text"] = test_df["tagged_in_context"].apply(preprocess_text)

# Tokenize the text data for test set
test_sequences = tokenizer.texts_to_sequences(test_df["text"])
test_data = pad_sequences(test_sequences, maxlen=max_length)

# Prepare the labels for test set
test_labels = np.where(test_df["label"] == "not_propaganda", 0, 1)

# Predict labels for test data
predictions = model.predict(test_data)
predicted_labels = np.round(predictions).astype(int).flatten()

# Map actual labels to labels for test set
actual_labels_mapped = ['not_propaganda' if label == 0 else 'propaganda' for label in test_labels]

# Map predicted values to labels for test set
predicted_labels_mapped = ['not_propaganda' if pred == 0 else 'propaganda' for pred in predicted_labels]

# Convert predicted labels and test labels to pandas Series
test_labels_series = pd.Series(actual_labels_mapped, name='Actual')
predicted_labels_series = pd.Series(predicted_labels_mapped, name='Predicted')

# Concatenate the actual and predicted labels into a DataFrame for test set
test_results_df = pd.concat([test_labels_series, predicted_labels_series], axis=1)

print(test_results_df.head(10))

# Classification report for test dataset
print(classification_report(test_labels, predicted_labels))


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
           Actual       Predicted
0  not_propaganda  not_propaganda
1      propaganda      propaganda
2      propaganda  not_propaganda
3  not_propaganda  not_propaganda
4      propaganda      propaganda
5      propaganda      propaganda
6      propaganda  not_propaganda
7  not_propaganda  not_propaganda
8      propaganda  not_propaganda
9      propaganda      propaganda
              precision    recall  f1-score   support

           0       0.74      0.95      0.83       301
           1       0.92      0.64      0.75       279

    accuracy                           0.80       580
   macro avg       0.83      0.79      0.79       580
weighted avg       0.82      0.80      0.79       580



## Task 2: Propaganda Technique Classification
### Approach 1: Fine-tuned BERT 

In [16]:
from transformers import BertTokenizer, BertModel, AdamW
from collections import Counter
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


# Calculate class weights
class_distribution = Counter(train_df["label"])
total_samples = sum(class_distribution.values())
class_weights = {cls: total_samples / (len(class_distribution) * count) for cls, count in class_distribution.items()}

idx_to_label = {0: 'not_propaganda', 1: 'exaggeration,minimisation', 2: 'causal_oversimplification',
                3: 'name_calling,labeling', 4: 'loaded_language', 5: 'appeal_to_fear_prejudice',
                6: 'flag_waving', 7: 'repetition', 8: 'doubt', 9: 'not_propaganda'}


# Define label-to-id and id-to-label mappings
label_to_id = {label: idx for idx, label in enumerate(class_distribution.keys())}
id_to_label = {idx: label for label, idx in label_to_id.items()}

# Preprocess the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocess the data
def tokenize_data(data, tokenizer, max_length):
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in data.iterrows():
        encoded_dict = tokenizer.encode_plus(
            row['tagged_in_context'],
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        labels.append(label_to_id[row['label']])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Tokenize the text data for train and validation sets
max_length = 128  # Adjust as needed
train_input_ids, train_attention_masks, train_labels = tokenize_data(train_data, tokenizer, max_length)
val_input_ids, val_attention_masks, val_labels = tokenize_data(val_data, tokenizer, max_length)

# Define DataLoader for training and validation sets
batch_size = 32  # 
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Define the model architecture
class BertForTC(nn.Module):
    def __init__(self, num_classes):
        super(BertForTC, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, 128)
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.fc(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize the model
num_classes = len(label_to_id)
model = BertForTC(num_classes)

# Set up the optimizer and loss function with class weights
optimizer = AdamW(model.parameters(), lr=2e-5)
class_weights_tensor = torch.tensor([class_weights[idx_to_label[i]] for i in range(num_classes)]).float().to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Train the model
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)

    for batch in progress_bar:
        input_ids, attention_masks, labels = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_masks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        progress_bar.set_postfix({'train_loss': train_loss / len(train_dataloader)})
        
    average_train_loss = train_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {average_train_loss:.4f}")

    # Evaluate the model on the validation set
    model.eval()
    val_preds = []
    val_true_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_masks, labels = batch
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_masks)
            _, predicted = torch.max(outputs, 1)

            val_preds.extend(predicted.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true_labels, val_preds)
    print(f"Validation Accuracy: {val_accuracy:.4f}")

    # Convert predicted labels from integers to technique names
    predicted_labels_mapped = [id_to_label[label] for label in val_preds]
    true_labels_mapped = [id_to_label[label] for label in val_true_labels]

    # Print classification report
    print(classification_report(true_labels_mapped, predicted_labels_mapped))

# Test the model on the test set
test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df, tokenizer, max_length)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

model.eval()
test_preds = []
test_true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_masks, labels = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)

        outputs = model(input_ids, attention_masks)
        _, predicted = torch.max(outputs, 1)

        test_preds.extend(predicted.cpu().numpy())
        test_true_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(test_true_labels, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Convert predicted labels from integers to technique names
predicted_labels_mapped = [id_to_label[label] for label in test_preds]
true_labels_mapped = [id_to_label[label] for label in test_true_labels]

# Print classification report for test set
print(classification_report(true_labels_mapped, predicted_labels_mapped))


Epoch 1/5 - Train Loss: 2.1671
Validation Accuracy: 0.4928
                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.26      0.38      0.31        29
causal_oversimplification       0.21      0.83      0.33        23
                    doubt       0.64      0.21      0.31        34
exaggeration,minimisation       0.15      0.09      0.11        35
              flag_waving       0.43      0.38      0.40        32
          loaded_language       0.00      0.00      0.00        39
    name_calling,labeling       0.11      0.26      0.16        23
           not_propaganda       0.81      0.74      0.77       241
               repetition       0.20      0.04      0.06        27

                 accuracy                           0.49       483
                macro avg       0.31      0.32      0.27       483
             weighted avg       0.53      0.49      0.49       483



Epoch 2/5 - Train Loss: 1.9083


/Users/maicymaritim/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/maicymaritim/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/maicymaritim/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

Validation Accuracy: 0.5135
                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.34      0.34      0.34        29
causal_oversimplification       0.40      0.26      0.32        23
                    doubt       0.40      0.29      0.34        34
exaggeration,minimisation       0.00      0.00      0.00        35
              flag_waving       0.32      0.53      0.40        32
          loaded_language       0.12      0.13      0.12        39
    name_calling,labeling       0.15      0.65      0.25        23
           not_propaganda       0.87      0.77      0.81       241
               repetition       0.00      0.00      0.00        27

                 accuracy                           0.51       483
                macro avg       0.29      0.33      0.29       483
             weighted avg       0.54      0.51      0.51       483



Epoch 3/5 - Train Loss: 1.5324
Validation Accuracy: 0.5880
                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.46      0.55      0.50        29
causal_oversimplification       0.37      0.74      0.49        23
                    doubt       0.67      0.12      0.20        34
exaggeration,minimisation       0.40      0.06      0.10        35
              flag_waving       0.52      0.50      0.51        32
          loaded_language       0.29      0.26      0.27        39
    name_calling,labeling       0.17      0.43      0.25        23
           not_propaganda       0.89      0.83      0.86       241
               repetition       0.22      0.37      0.28        27

                 accuracy                           0.59       483
                macro avg       0.44      0.43      0.38       483
             weighted avg       0.64      0.59      0.59       483



Epoch 4/5 - Train Loss: 1.1777
Validation Accuracy: 0.5921
                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.34      0.55      0.42        29
causal_oversimplification       0.31      0.96      0.46        23
                    doubt       0.50      0.09      0.15        34
exaggeration,minimisation       0.67      0.17      0.27        35
              flag_waving       0.55      0.50      0.52        32
          loaded_language       0.38      0.26      0.31        39
    name_calling,labeling       0.33      0.43      0.38        23
           not_propaganda       0.93      0.80      0.86       241
               repetition       0.18      0.37      0.24        27

                 accuracy                           0.59       483
                macro avg       0.47      0.46      0.40       483
             weighted avg       0.68      0.59      0.60       483



Epoch 5/5 - Train Loss: 0.8490
Validation Accuracy: 0.6522
                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.38      0.62      0.47        29
causal_oversimplification       0.40      0.61      0.48        23
                    doubt       0.48      0.38      0.43        34
exaggeration,minimisation       0.44      0.49      0.46        35
              flag_waving       0.50      0.69      0.58        32
          loaded_language       0.44      0.18      0.25        39
    name_calling,labeling       0.27      0.52      0.35        23
           not_propaganda       0.97      0.86      0.91       241
               repetition       0.33      0.19      0.24        27

                 accuracy                           0.65       483
                macro avg       0.47      0.50      0.46       483
             weighted avg       0.69      0.65      0.66       483

Test Accuracy: 0.6517
                           precision    recal

#### Model Testing

In [17]:
# Tokenize the text data
max_length = 128  #
test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df, tokenizer, max_length)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Test the model on the test set
model.eval()
test_preds = []
test_true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_masks, labels = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)

        outputs = model(input_ids, attention_masks)
        _, predicted = torch.max(outputs, 1)

        test_preds.extend(predicted.cpu().numpy())
        test_true_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(test_true_labels, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Convert predicted labels from integers to technique names
predicted_labels_mapped = [id_to_label[label] for label in test_preds]
true_labels_mapped = [id_to_label[label] for label in test_true_labels]

# Print classification report for test set
print(classification_report(true_labels_mapped, predicted_labels_mapped))

# Create a DataFrame for actual vs. predicted labels
test_results_df = pd.DataFrame({'Actual': true_labels_mapped, 'Predicted': predicted_labels_mapped})

# Print the first 10 rows of the DataFrame
print(test_results_df.head(10))


Test Accuracy: 0.6517
                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.47      0.67      0.55        43
causal_oversimplification       0.45      0.45      0.45        31
                    doubt       0.49      0.61      0.54        38
exaggeration,minimisation       0.21      0.32      0.25        28
              flag_waving       0.47      0.69      0.56        39
          loaded_language       0.25      0.14      0.18        37
    name_calling,labeling       0.44      0.45      0.44        31
           not_propaganda       0.95      0.83      0.89       301
               repetition       0.27      0.19      0.22        32

                 accuracy                           0.65       580
                macro avg       0.44      0.48      0.45       580
             weighted avg       0.68      0.65      0.66       580

                      Actual                  Predicted
0             not_propaganda             not_pro

### Approach 2: CNN-BiLSTM for Propaganda Techniques Classification

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

# Define propaganda techniques
propaganda_techniques = ['not_propaganda', 'exaggeration,minimisation', 'causal_oversimplification',
                         'name_calling,labeling', 'loaded_language', 'appeal_to_fear_prejudice',
                         'flag_waving', 'repetition', 'doubt']

# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_df["tagged_in_context"])
train_sequences = tokenizer.texts_to_sequences(train_df["tagged_in_context"])
test_sequences = tokenizer.texts_to_sequences(test_df["tagged_in_context"])

# Pad the sequences to a fixed length
max_length = 100
train_data = pad_sequences(train_sequences, maxlen=max_length)
test_data = pad_sequences(test_sequences, maxlen=max_length)

# Manually map the labels to integers
label_to_id = {label: idx for idx, label in enumerate(propaganda_techniques)}
train_labels = train_df["label"].map(label_to_id)
test_labels = test_df["label"].map(label_to_id)

# Convert the labels to numpy arrays
train_labels = train_labels.to_numpy()
test_labels = test_labels.to_numpy()

# Split the data into train and validation sets (80:20 ratio)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Calculate class weights
class_weights = {}
for i, technique in enumerate(propaganda_techniques):
    class_weights[i] = len(train_labels) / (len(propaganda_techniques) * (train_labels == i).sum())

# Define the CNN-BiLSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(propaganda_techniques), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with class weights
history = model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(val_data, val_labels), class_weight=class_weights)

# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels, batch_size=32)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predictions
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)

# Convert predicted labels to technique names
predicted_labels_mapped = [propaganda_techniques[label] for label in predicted_labels]

# Convert true labels to technique names
true_labels_mapped = [propaganda_techniques[label] for label in test_labels]

# Classification report
print(classification_report(true_labels_mapped, predicted_labels_mapped))

# Create a DataFrame to store actual vs predicted labels during validation
val_predictions = model.predict(val_data)
val_predicted_labels = np.argmax(val_predictions, axis=1)
val_predicted_labels_mapped = [propaganda_techniques[label] for label in val_predicted_labels]

# Map numerical labels to their corresponding propaganda techniques
actual_labels_mapped = [propaganda_techniques[label] for label in val_labels]

#Create a DataFrame to store actual vs predicted labels during validation
result_df = pd.DataFrame({'Actual_Label': actual_labels_mapped, 'Predicted_Label': val_predicted_labels_mapped})
print(result_df.head(25))

/Users/maicymaritim/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 15s 81ms/step - accuracy: 0.0734 - loss: 2.2006 - val_accuracy: 0.0683 - val_loss: 2.2149
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.1190 - loss: 2.1610 - val_accuracy: 0.1180 - val_loss: 2.2041
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.2477 - loss: 2.0325 - val_accuracy: 0.3996 - val_loss: 1.9495
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.5199 - loss: 1.4871 - val_accuracy: 0.3168 - val_loss: 2.1111
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.7095 - loss: 0.8920 - val_accuracy: 0.3892 - val_loss: 1.9457
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.8541 - loss: 0.5104 - val_accuracy: 0.3851 - val_loss: 2.4308
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.9027 - loss: 0.3727 - val_accuracy: 0.3996 - val_loss: 2.6493
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.9473 - loss: 0.2354 - val_accuracy: 0.4203 - 

#### Model Testing

In [19]:
# Predictions on test data
test_predictions = model.predict(test_data)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_predicted_labels_mapped = [propaganda_techniques[label] for label in test_predicted_labels]

# Convert true labels to technique names
test_true_labels_mapped = [propaganda_techniques[label] for label in test_labels]

# Classification report for test set
print(classification_report(test_true_labels_mapped, test_predicted_labels_mapped))

# Create a DataFrame to store actual vs predicted labels for test set
test_result_df = pd.DataFrame({'Actual_Label': test_true_labels_mapped, 'Predicted_Label': test_predicted_labels_mapped})

# Display first 10 rows of the DataFrame
print(test_result_df.head(10))

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.33      0.07      0.12        43
causal_oversimplification       0.18      0.35      0.24        31
                    doubt       0.26      0.24      0.25        38
exaggeration,minimisation       0.13      0.21      0.16        28
              flag_waving       0.55      0.46      0.50        39
          loaded_language       0.09      0.11      0.10        37
    name_calling,labeling       0.12      0.03      0.05        31
           not_propaganda       0.75      0.71      0.73       301
               repetition       0.24      0.41      0.30        32

                 accuracy                           0.48       580
                macro avg       0.29      0.29      0.27       580
             weighted avg       0.51      0.48      0.48       580

                Actual_Label        Predicted_Label
0             not_propaganda      